Visual Odometry (VO)

You will use the pykitti module and KITTI odometry dataset.

Download the odometry data from [here](https://drive.google.com/file/d/1Vbom0TPDB-NIkqrqsfUuGvDMi2S08uEt/view?usp=sharing).

## Monocular VO with OpenCV on KITTI

For each consecutive frame pair in the sequence, you will compute the relative pose between the frames and visualize it. You will use:

* pykitti code similar to what you wrote in Week 3 to load the seqeunce with ground-truth info. (Check out the [demo code](https://github.com/utiasSTARS/pykitti/blob/master/demos/demo_odometry.py))
* OpenCV functions to compute and visualize the features and the essential matrix.

Please follow these steps to complete the assignment:

1. You can use the ORB Feature to do the feature matching:
    `orb = cv2.ORB_create()` to create the ORB object
    and then `orb.detectAndCompute()` to find the keypoints and descriptors on both frames

2. You can use brute-force matcher to match ORB descriptors:
    `bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)`

3. After matching the descriptors, sort the matched keypoints.

4. Draw matches on the two images using the `cv2.drawMatches()` function.

5. Compute the essential matrix using the `cv2.findEssentialMat()` function. Note that you need the matching points and the instrinsics for this function. 

6. Extract the rotation and translation from the essential matrix using the `cv2.recoverPose()` function.

7. Multiply the estimated rotation and translation with the previous rotation and translation. Initialize rotation to identity and translation to zeros on the first frame.

8. Display the current image with the keypoints on it using the `cv2.drawKeypoints()` function.

9. Update the previous rotation and translation as the current rotation and translation.

10. Draw the estimated trajectory as blue and ground-truth trajectory as green. You can use the `cv2.circle()` function.


You can create a video of your visualization of images and poses for the provided sequence.


Some examples repositories that might be useful:
* https://bitbucket.org/castacks/visual_odometry_tutorial/src/master/visual-odometry/
* https://github.com/uoip/monoVO-python


In [ ]:
# !cd KITTI_odometry
import numpy as np

In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir(r'/content/drive/MyDrive/monocular-vo/KITTI_odometry/')

In [ ]:
!pwd

In [ ]:
import os 
os.listdir('sequences/09')

In [ ]:
path_dir = 'sequences/09/image_2'
paths = [p for p in os.listdir(path_dir) if p.endswith('.png')]
paths = sorted(paths)

# path 
print(len(paths))

In [ ]:
# !pwd
# paths

In [ ]:
# getting these info from the demo code - hyperparams for KITTI dataset
width = 1241.0
height = 376.0
fx, fy, cx, cy = [718.8560, 718.8560, 607.1928, 185.2157]
trajMap = np.zeros((1000, 1000, 3), dtype=np.uint8)


# gt trajectories 
gt_Traj = []
with open('poses/09.txt') as f:
    for line in f:
        arr = list(map(float, line.split(' ')))
        gt_Traj.append(np.array(arr).reshape(3, 4))


In [ ]:
gt_Traj[0]

In [ ]:
import matplotlib.pyplot as plt
import cv2


In [ ]:

# os.path.join(path_dir,paths[0])
# paths

In [ ]:
from google.colab.patches import cv2_imshow # if you are running on the collab if not use cv2.show()
# I use one of the codes suggested in the tutorials.  
import cv2
prev_img = cv2.imread(os.path.join(path_dir, paths[0]), 0)
for i, path in enumerate(paths[1:]):
    cur_img = cv2.imread(os.path.join(path_dir, path), 0)
    # feature matching according to the demo code 
    orb = cv2.ORB_create(nfeatures=6000)

    kp1, des1 = orb.detectAndCompute(prev_img, None)
    kp2, des2 = orb.detectAndCompute(cur_img, None)
    # use brute-force matcher
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(des1, des2)
    matches = sorted(matches, key = lambda x:x.distance)


    img_matching = cv2.drawMatches(cur_img, kp1, prev_img, kp2, matches[0:100], None)
    # cv2.imshow('feature matching', img_matching) # if u are running on Jupyter
    cv2_imshow(img_matching) # if you are running on colab
    pts1 = np.float32([kp1[m.queryIdx].pt for m in matches])
    pts2 = np.float32([kp2[m.trainIdx].pt for m in matches])
    # calculating the Essential matrix 
    E, mask = cv2.findEssentialMat(pts1, pts2, focal=fx, pp=(cx, cy), method=cv2.RANSAC, prob=0.999, threshold=1)
    pts1 = pts1[mask.ravel() == 1]
    pts2 = pts2[mask.ravel() == 1]

    _, R, t, mask = cv2.recoverPose(E, pts1, pts2, focal=fx, pp=(cx, cy))

    # get camera motion
    R = R.transpose()
    t = np.matmul(R, t)

    if i ==0 : 
        curr_R, curr_t = R, t
    else:
        curr_R, curr_t = np.matmul(prev_R, R), np.matmul(prev_R, t) + prev_t

    # draw the current image with keypoints
    curr_img_kp = cv2.drawKeypoints(cur_img, kp2, None, color=(0, 255, 0), flags=0)
    # cv2.imshow('keypoints from current image', curr_img_kp) # if u are running on Jupyter
    cv2_imshow(curr_img_kp) # if you are running on colab 
    # draw estimated trajectory (blue) and gt trajectory (red)
    offset_draw = (int(1000/2))
    cv2.circle(trajMap, (-int(curr_t[0])+offset_draw, int(curr_t[2])+offset_draw), 1, (255,0,0), 2)
    cv2.circle(trajMap, (int(gt_Traj[i][0, 3])+offset_draw, -int(gt_Traj[i][2, 3])+offset_draw), 1, (0,0,255), 2)
    # cv2.imshow('Trajectory', trajMap) # if u are running on Jupyter
    cv2_imshow(trajMap) # if you are running on colab 
    cv2.waitKey(1)

    prev_R, prev_t = curr_R, curr_t
    prev_img = cur_img
    
cv2.imwrite('trajMap.png', trajMap)